In [24]:
import pandas as pd
import numpy as np

In [2]:
ssmm_candidates_x_votes_info_conv_1_11 = pd.read_csv('ssmm_candidates_x_votes_info_conv_1_11.csv', sep='\t', encoding='utf-8')

ssmm_candidates_x_votes_info_conv_1_11.head()


,conv_x_name,name,convocation_no,year_stream_no,academic_group_no,program_txt,announcement_link,announcement_dttm,votes_cnt,elected_flg,result_link,result_dttm,appearance_dttm
0,1_Левин_Андрей,Левин Андрей,1,1.1,101.0,Активный участник и организатор различных меро...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,48.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
1,1_Матвеев_Дмитрий_Александрович,Матвеев Дмитрий Александрович,1,1.1,101.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,49.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
2,1_Власов_Артем_Андреевич,Власов Артем Андреевич,1,1.2,107.0,NaN,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,36.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
3,1_Кириллова_Полина,Кириллова Полина,1,1.2,112.0,Мне нравится заниматься общественной деятельно...,https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,40.0,1,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00
4,1_Семенов_Павел,Семенов Павел,1,1.2,112.0,"""Здравствуйте. Я так понимаю, что я уже поздно...",https://vk.com/topic-33327655_26089953,2012-02-26 00:16:00,20.0,0,https://vk.com/topic-33327655_26089953,2012-03-07 20:55:00,2012-02-26 00:16:00


## SSMM_ID

In [3]:
# СТЕРЖНЕВАЯ ТАБЛИЦА КАНДИДАТОВ С ПОРЯДКОВЫМ ssmm_id

# упорядочивание по времени первого появления, при равенстве - по ФИО
full_min_dttm = ssmm_candidates_x_votes_info_conv_1_11.groupby(['name']).agg({'appearance_dttm': ['min','max']}).reset_index()
full_min_dttm.columns = ['_'.join(col[::-1]) for col in full_min_dttm.columns.values]
full_min_dttm = full_min_dttm.rename(columns={'_name':'name'}).sort_values(['min_appearance_dttm', 'name'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)

full_min_dttm

import numpy as np

full_min_dttm['index'] = np.arange(full_min_dttm.shape[0])
full_min_dttm['ssmm_id'] = full_min_dttm['index'].apply(lambda x: x+1)
ssmm_candidates_core = full_min_dttm.drop(['index'], axis=1)
ssmm_candidates_core = ssmm_candidates_core[['ssmm_id', 'name', 'min_appearance_dttm', 'max_appearance_dttm']].rename(columns={'name':'candidate_name','min_appearance_dttm':'first_appearance_dttm','max_appearance_dttm':'last_appearance_dttm'})

ssmm_candidates_core
# ssmm_candidates_core[:60]


,ssmm_id,candidate_name,first_appearance_dttm,last_appearance_dttm
0,1,Владыкина Вероника Евгеньевна,2012-02-25 23:16:00,2013-11-17 14:21:00
1,2,Коновалов Антон Александрович,2012-02-25 23:16:00,2012-11-04 23:03:00
2,3,Ласкин Алексей Александрович,2012-02-25 23:16:00,2013-11-17 14:21:00
3,4,Малхасян Арамаис,2012-02-25 23:16:00,2012-02-25 23:16:00
4,5,Попов Леонид Андреевич,2012-02-25 23:16:00,2013-11-17 14:21:00
...,...,...,...,...
315,316,Уваров Андрей Михайлович,2021-10-15 11:11:00,2021-10-15 11:11:00
316,317,Поликарпов Александр Валерьевич,2021-10-16 10:50:00,2021-10-16 10:50:00
317,318,Каляев Тимур Джанбулатович,2021-10-16 11:20:00,2021-10-16 11:20:00
318,319,Маммагаджиев Руслан Гаджимагомедович,2021-10-16 11:20:00,2021-10-16 11:20:00


In [4]:
ssmm_candidates_core.to_csv('ssmm_candidates_core.csv', sep='\t', encoding='utf-8', index=False)


In [5]:
# member_id - номер кандидата в порядке избрания в члены

temd_df = ssmm_candidates_x_votes_info_conv_1_11
elected_min_dttm = temd_df[temd_df['elected_flg'] == 1].groupby(['name']).agg({"result_dttm": "min"}).reset_index().sort_values(['result_dttm', 'name'],
              ascending = [True, True]).reset_index().drop(['index'], axis=1)

import numpy as np

elected_min_dttm['index'] = np.arange(elected_min_dttm.shape[0])
elected_min_dttm['member_id'] = elected_min_dttm['index'].apply(lambda x: x+1)
ssmm_members_core = elected_min_dttm.drop(['index'], axis=1)
ssmm_members_core = ssmm_members_core[['member_id', 'name', 'result_dttm']].rename(columns={'name':'member_name','result_dttm':'first_result_dttm'})

# ssmm_members_core
ssmm_members_core #[:30]


,member_id,member_name,first_result_dttm
0,1,Аканаев Тимур Рашитович,2012-02-28 22:34:00
1,2,Асташов Дмитрий Сергеевич,2012-02-28 22:34:00
2,3,Гильманов Роман Павлович,2012-02-28 22:34:00
3,4,Глухова Алёна Олеговна,2012-02-28 22:34:00
4,5,Костицын Александр Алексеевич,2012-02-28 22:34:00
...,...,...,...
258,259,Каляев Тимур Джанбулатович,2021-10-20 20:21:00
259,260,Маммагаджиев Руслан Гаджимагомедович,2021-10-20 20:21:00
260,261,Третьякова Екатерина Алексеевна,2021-10-20 20:21:00
261,262,Терехов Михаил Сергеевич,2021-10-21 15:44:00


## To create new tables-DICTIONARIES

In [8]:
def define_classic_year_no(year_stream_no):
    if 'm' in year_stream_no:
        classic_year_no = year_stream_no
    else:
        classic_year_no = year_stream_no[0]
    return classic_year_no

# define_classic_year_no('2.1')

def define_specialization(stream_no):
    if stream_no in [1,2]:
        spec = 'Фундаментальная математика'
    elif stream_no == 3:
        spec = 'Фундаментальная механика'
    elif stream_no == 4:
        spec = 'Математика и экономическая теория' # старое: Математические методы экономики
    elif stream_no == 5:
        spec = 'Фундаментальная математика и математическая физика'
    elif stream_no == 0:
        spec = 'Магистратура'
    else:
        spec = 'ERROR'
    return spec

def define_short_specialization(stream_no):
    if stream_no in [1,2]:
        spec = 'Математика'
    elif stream_no == 3:
        spec = 'Механика'
    elif stream_no == 4:
        spec = 'Эконом' # старое: Математические методы экономики
    elif stream_no == 5:
        spec = 'ФММФ'
    elif stream_no == 0:
        spec = 'Магистратура'
    else:
        spec = 'ERROR'
    return spec

# С сентября 2021 года формируется группа 241 - экспериментальная группа программы (специализации) 
# "Фундаментальная математика и математическая физика".
# О программе можно посмотреть на сайте https://fmmp.math.msu.ru/about/

# учебные планы
# https://math.msu.ru/uplans


def define_course_level(year_stream_no):
    if 'm' in year_stream_no:
        course_level = 'Старшие курсы'
    elif int(year_stream_no[0]) <= 2:
        course_level = 'Младшие курсы'
    elif int(year_stream_no[0]) >= 3:
        course_level = 'Старшие курсы'
    else:
        course_level = 'ERROR'
    return course_level

def split_year_stream(year_stream_no):
    if 'm' in year_stream_no:
        year_no = int(year_stream_no[-1])
        stream_no = 0
    elif '.' in year_stream_no:
        year_no = int(year_stream_no[0])
        stream_no = int(year_stream_no[-1])
    else:
        year_no, stream_no = -1, -1
    return year_no, stream_no

# split_year_stream('m2')


In [9]:

ssmm_dict_year_stream = ssmm_candidates_x_votes_info_conv_1_11[['year_stream_no']].drop_duplicates().sort_values(['year_stream_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)
# ssmm_dict_year_stream['kek'] = ssmm_dict_year_stream['year_stream_no'].apply(lambda x: list(split_year_stream(x)))
ssmm_dict_year_stream['classic_year_no'] = ssmm_dict_year_stream['year_stream_no'].apply(lambda x: define_classic_year_no(x))
ssmm_dict_year_stream[['year_no','stream_no']] = pd.DataFrame(ssmm_dict_year_stream['year_stream_no'].apply(lambda x: list(split_year_stream(x))).tolist(), index=ssmm_dict_year_stream.index)
ssmm_dict_year_stream['course_level'] = ssmm_dict_year_stream['year_stream_no'].apply(lambda x: define_course_level(x))
ssmm_dict_year_stream['specialization'] = ssmm_dict_year_stream['stream_no'].apply(lambda x: define_specialization(x))
ssmm_dict_year_stream['short_specialization'] = ssmm_dict_year_stream['stream_no'].apply(lambda x: define_short_specialization(x))
ssmm_dict_year_stream

# df2[['team1','team2']] = pd.DataFrame(df2.teams.tolist(), index= df2.index)


,year_stream_no,classic_year_no,year_no,stream_no,course_level,specialization,short_specialization
0,1.1,1,1,1,Младшие курсы,Фундаментальная математика,Математика
1,1.2,1,1,2,Младшие курсы,Фундаментальная математика,Математика
2,1.3,1,1,3,Младшие курсы,Фундаментальная механика,Механика
3,2.1,2,2,1,Младшие курсы,Фундаментальная математика,Математика
4,2.2,2,2,2,Младшие курсы,Фундаментальная математика,Математика
5,2.3,2,2,3,Младшие курсы,Фундаментальная механика,Механика
6,2.5,2,2,5,Младшие курсы,Фундаментальная математика и математическая фи...,ФММФ
7,3.1,3,3,1,Старшие курсы,Фундаментальная математика,Математика
8,3.2,3,3,2,Старшие курсы,Фундаментальная математика,Математика
9,3.3,3,3,3,Старшие курсы,Фундаментальная механика,Механика


In [49]:
ssmm_dict_year_stream.to_csv('ssmm_dict_year_stream.csv', sep='\t', encoding='utf-8', index=False)
# ssmm_dict_year_stream.to_csv('./pulling original data/ssmm_dict_year_stream.csv', sep='\t', encoding='utf-8', index=False)
# ssmm_dict_year_stream.to_csv('../home/ssmm_dict_year_stream.csv', sep='\t', encoding='utf-8', index=False)


OSError: Cannot save file into a non-existent directory: 'home'

In [11]:
def define_graduated_year(year_stream_no, year):
    if 'm' in year_stream_no:
        if year_stream_no == 'm2':
            graduated_flg = 1
        else:
            graduated_flg = 0
    elif int(year_stream_no[0]) == 5 and year < 2016:
        graduated_flg = 1
    elif int(year_stream_no[0]) == 6 and year >= 2016:
        graduated_flg = 1
    else:
        graduated_flg = 0
    return graduated_flg

def year_by_conv_no(conv_no):
    if conv_no >= 2:
        return conv_no+2010
    elif conv_no == 1:
        return 2012
    else:
        return -1


In [12]:
ssmm_dict_years = ssmm_candidates_x_votes_info_conv_1_11[['convocation_no','year_stream_no']].drop_duplicates().sort_values(['convocation_no','year_stream_no'],
              ascending = [True,True]).reset_index().drop(['index'], axis=1)
ssmm_dict_years['year'] = ssmm_dict_years['convocation_no'].apply(lambda x: year_by_conv_no(x) )
ssmm_dict_years['graduated_flg'] = ssmm_dict_years.apply(lambda x: define_graduated_year(x['year_stream_no'], x['year']), axis=1)

ssmm_dict_graduated_years = ssmm_dict_years
ssmm_dict_graduated_years


,convocation_no,year_stream_no,year,graduated_flg
0,1,1.1,2012,0
1,1,1.2,2012,0
2,1,1.3,2012,0
3,1,2.1,2012,0
4,1,2.2,2012,0
...,...,...,...,...
194,11,5.3,2021,0
195,11,5.4,2021,0
196,11,6.1,2021,1
197,11,6.3,2021,1


In [13]:
ssmm_dict_graduated_years.to_csv('ssmm_dict_graduated_years.csv', sep='\t', encoding='utf-8', index=False)


## ФЛАГИ КАНДИДАТОВ

In [14]:
# флаги о членстве
# - сколько раз избирался
# - первый созыв
# - последний созыв
# - список созывов

temd_df = ssmm_candidates_x_votes_info_conv_1_11
ssmm_members_x_conv = temd_df[temd_df['elected_flg'] == 1].sort_values(['convocation_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)

ssmm_members_conv = ssmm_members_x_conv.groupby('name').agg({'convocation_no':['min','max', lambda x: list(x), lambda x: len(list(x))]}) 

ssmm_members_conv.columns = ['_'.join(col[::-1]) for col in ssmm_members_conv.columns.values]
ssmm_members_conv = ssmm_members_conv.reset_index().rename(columns={'<lambda_0>_convocation_no':'convocation_list','<lambda_1>_convocation_no':'convocation_cnt'})
ssmm_members_conv = ssmm_members_conv.sort_values(['convocation_cnt', 'min_convocation_no'],
              ascending = [False, True]).reset_index().drop(['index'], axis=1)
ssmm_members_conv[:60]



,name,min_convocation_no,max_convocation_no,convocation_list,convocation_cnt
0,Денисов Илья Олегович,3,8,"[3, 4, 5, 6, 7, 8]",6
1,Краснова Анастасия Александровна,4,9,"[4, 5, 6, 7, 8, 9]",6
2,Муравьёва Анна Сергеевна,4,9,"[4, 5, 6, 7, 8, 9]",6
3,Мазитова Вероника Рустэмовна,6,11,"[6, 7, 8, 9, 10, 11]",6
4,Матвеев Дмитрий Александрович,1,5,"[1, 2, 3, 4, 5]",5
5,Остроухова Наталья Владимировна,2,6,"[2, 3, 4, 5, 6]",5
6,Савченко Анастасия Игоревна,3,7,"[3, 4, 5, 6, 7]",5
7,Малахов Николай Сергеевич,6,10,"[6, 7, 8, 9, 10]",5
8,Малиновский Георгий Андреевич,7,11,"[7, 8, 9, 10, 11]",5
9,Матвеев Алексей Викторович,1,4,"[1, 2, 3, 4]",4


In [15]:
ssmm_members_conv.to_csv('ssmm_members_conv.csv', sep='\t', encoding='utf-8', index=False)


In [18]:
# то же самое, но для выдвижения, а не членства

temd_df = ssmm_candidates_x_votes_info_conv_1_11
ssmm_candidates_x_conv = temd_df.sort_values(['convocation_no'],
              ascending = [True]).reset_index().drop(['index'], axis=1)

ssmm_candidates_conv = ssmm_candidates_x_conv.groupby('name').agg({'convocation_no':['min','max', lambda x: list(x), lambda x: len(list(x))]}) 

ssmm_candidates_conv.columns = ['_'.join(col[::-1]) for col in ssmm_candidates_conv.columns.values]
ssmm_candidates_conv = ssmm_candidates_conv.reset_index().rename(columns={'<lambda_0>_convocation_no':'convocation_list','<lambda_1>_convocation_no':'convocation_cnt'})
ssmm_candidates_conv = ssmm_candidates_conv.sort_values(['convocation_cnt', 'min_convocation_no'],
              ascending = [False, True]).reset_index().drop(['index'], axis=1)
ssmm_candidates_conv.head()



,name,min_convocation_no,max_convocation_no,convocation_list,convocation_cnt
0,Денисов Илья Олегович,3,8,"[3, 4, 5, 6, 7, 8]",6
1,Краснова Анастасия Александровна,4,9,"[4, 5, 6, 7, 8, 9]",6
2,Муравьёва Анна Сергеевна,4,9,"[4, 5, 6, 7, 8, 9]",6
3,Мазитова Вероника Рустэмовна,6,11,"[6, 7, 8, 9, 10, 11]",6
4,Матвеев Дмитрий Александрович,1,5,"[1, 2, 3, 4, 5]",5


In [19]:
# НА КАКИХ КУРСАХ ВЫДВИГАЮТСЯ?
# - вообще
# - впервые
# - в последний раз
# - сколько раз подавался
# - на каком курсе впервые подался (нужно тянуть номер курса)
# - на каком курсе в последний раз подался (нужно тянуть номер курса)

temp_df = ssmm_candidates_x_votes_info_conv_1_11

temp_df[['year_no','stream_no']] = pd.DataFrame(temp_df['year_stream_no'].apply(lambda x: list(split_year_stream(x))).tolist(), index=temp_df.index)
temp_df = temp_df.merge(ssmm_candidates_core, left_on='name', right_on='candidate_name',how='left')

first_temp_df = temp_df[temp_df['appearance_dttm'] == temp_df['first_appearance_dttm']]
first_temp_df = first_temp_df[['name','year_no','year_stream_no']].rename(columns={'year_no':'year_no_of_first_application'})

last_temp_df = temp_df[temp_df['appearance_dttm'] == temp_df['last_appearance_dttm']]
last_temp_df = last_temp_df[['name','year_no','year_stream_no']].rename(columns={'year_no':'year_no_of_last_application'})

dfList = [ssmm_candidates_conv,first_temp_df,last_temp_df]

from functools import reduce
ssmm_candidates_conv_2 = reduce(lambda df1,df2: pd.merge(df1,df2,on='name'), dfList)

ssmm_candidates_conv_2[:40]



,name,min_convocation_no,max_convocation_no,convocation_list,convocation_cnt,year_no_of_first_application,year_stream_no_x,year_no_of_last_application,year_stream_no_y
0,Денисов Илья Олегович,3,8,"[3, 4, 5, 6, 7, 8]",6,1,1.2,6,6.2
1,Краснова Анастасия Александровна,4,9,"[4, 5, 6, 7, 8, 9]",6,1,1.2,6,6.4
2,Муравьёва Анна Сергеевна,4,9,"[4, 5, 6, 7, 8, 9]",6,1,1.3,6,6.3
3,Мазитова Вероника Рустэмовна,6,11,"[6, 7, 8, 9, 10, 11]",6,1,1.1,6,6.1
4,Матвеев Дмитрий Александрович,1,5,"[1, 2, 3, 4, 5]",5,1,1.1,5,5.1
5,Остроухова Наталья Владимировна,2,6,"[2, 3, 4, 5, 6]",5,1,1.1,5,5.1
6,Савченко Анастасия Игоревна,3,7,"[3, 4, 5, 6, 7]",5,2,2.3,6,6.3
7,Малахов Николай Сергеевич,6,10,"[6, 7, 8, 9, 10]",5,2,2.2,6,6.1
8,Малиновский Георгий Андреевич,7,11,"[7, 8, 9, 10, 11]",5,1,1.1,5,5.4
9,Матвеев Алексей Викторович,1,4,"[1, 2, 3, 4]",4,2,2.1,5,5.1


In [20]:
ssmm_candidates_conv_2.to_csv('ssmm_candidates_conv.csv', sep='\t', encoding='utf-8', index=False)


In [22]:

# - сделать таблицу-матрицу с 11 колонками созывов и флагами членства

temd_df = ssmm_candidates_x_votes_info_conv_1_11[ssmm_candidates_x_votes_info_conv_1_11['elected_flg'] == 1]
# .sort_values(['convocation_cnt', 'min_convocation_no'],
#               ascending = [False, True]).reset_index().drop(['index'], axis=1)

list_for_sort = []
for i in range(1,11+1):
    list_for_sort.append('conv_'+str(i))

# просто флаг
pivoted = temd_df.pivot(index="name", columns="convocation_no", values="elected_flg").reset_index().fillna(0)

pivoted = pivoted.add_prefix('conv_').sort_values(list_for_sort+['conv_name'],
              ascending = [False, False, False, False, False, False, False, False, False, False, False, True]).rename(columns={'conv_name':'name'}).reset_index().drop(['index'], axis=1)

for i in range(1,11+1):
    pivoted['conv_'+str(i)] = pivoted['conv_'+str(i)].apply(lambda x: int(x) )
pivoted.head()


convocation_no,name,conv_1,conv_2,conv_3,conv_4,conv_5,conv_6,conv_7,conv_8,conv_9,conv_10,conv_11
0,Матвеев Дмитрий Александрович,1,1,1,1,1,0,0,0,0,0,0
1,Матвеев Алексей Викторович,1,1,1,1,0,0,0,0,0,0,0
2,Владыкина Вероника Евгеньевна,1,1,1,0,0,0,0,0,0,0,0
3,Осин Руслан Владимирович,1,1,1,0,0,0,0,0,0,0,0
4,Попов Леонид Андреевич,1,1,1,0,0,0,0,0,0,0,0


In [35]:
def is_not_zero(x):
    if x != '0':
        return 0
    else:
        return 1

# is_not_nan('1')

In [39]:
# - сделать таблицу-матрицу с 11 колонками созывов и НОМЕРОМ КУРСА в качестве флага членства


temd_df = ssmm_candidates_x_votes_info_conv_1_11[ssmm_candidates_x_votes_info_conv_1_11['elected_flg'] == 1].merge(ssmm_dict_year_stream[['year_stream_no','classic_year_no']],left_on='year_stream_no',right_on='year_stream_no',how='left')

# для удобства: отсеивать по числу созывов
min_convocation_cnt = 1
'''
ПРИ ВЫГРУЗКЕ ФАЙЛА ПАРАМЕТРУ ПРИСВОИТЬ ЗНАЧЕНИЕ == 1
'''

pivoted = temd_df.pivot(index="name", columns="convocation_no", values="classic_year_no").reset_index() #.fillna('0')
pivoted = pivoted.merge(ssmm_members_conv[['name','convocation_cnt']][ssmm_members_conv['convocation_cnt'] >= min_convocation_cnt], left_on='name', right_on='name',how='inner').sort_values(['name'],
              ascending = [True]).reset_index().drop(['index'], axis=1)

# pivoted = pivoted.add_prefix('conv_').sort_values(list_for_sort+['conv_name'],
#               ascending = [False, False, False, False, False, False, False, False, False, False, False, True]).rename(columns={'conv_name':'name'}).reset_index().drop(['index'], axis=1)

# key=lambda x: np.sign(x) -- работало для численного номера курса
pivoted = pivoted.add_prefix('conv_').sort_values(by=list_for_sort, key=lambda x: x.notna(),
              ascending = [False, False, False, False, False, False, False, False, False, False, False]).rename(columns={'conv_name':'name','conv_convocation_cnt':'convocation_cnt'}).reset_index().drop(['index'], axis=1)

# for i in range(1,11+1):
#     pivoted['conv_'+str(i)] = pivoted['conv_'+str(i)].apply(lambda x: int(x) )

ssmm_members_x_convocations = pivoted
ssmm_members_x_convocations[:60]

# ручная сортировка
# df.sort_values(by=['m'], key=lambda x: x.map(custom_dict))


,name,conv_1,conv_2,conv_3,conv_4,conv_5,conv_6,conv_7,conv_8,conv_9,conv_10,conv_11,convocation_cnt
0,Матвеев Дмитрий Александрович,1,2,3,4,5,NaN,NaN,NaN,NaN,NaN,NaN,5
1,Матвеев Алексей Викторович,2,3,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4
2,Владыкина Вероника Евгеньевна,3,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
3,Осин Руслан Владимирович,2,3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
4,Попов Леонид Андреевич,3,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
5,Сапунов Кирилл Вячеславович,3,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
6,Тропин Александр Михайлович,3,4,NaN,NaN,NaN,m1,NaN,NaN,NaN,NaN,NaN,3
7,Абдуллаева Эмилия Табриз кызы,4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
8,Асташов Дмитрий Сергеевич,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
9,Власов Артем Андреевич,1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2


In [40]:
ssmm_members_x_convocations.to_csv('ssmm_members_x_convocations.csv', sep='\t', encoding='utf-8', index=False)


In [43]:
# научиться выделять из этой таблицы блоки однокурсников (по состоянию на конкретный созыв)

parameter_conv_no = 4
parameter_classic_year_no = '3'

temp_df = ssmm_members_x_convocations
temp_df = temp_df[temp_df['conv_'+str(parameter_conv_no)] == parameter_classic_year_no] #.sort_values(['conv_4'],
#               ascending = [False]).reset_index().drop(['index'], axis=1)

temp_df

,name,conv_1,conv_2,conv_3,conv_4,conv_5,conv_6,conv_7,conv_8,conv_9,conv_10,conv_11,convocation_cnt
40,Остроухова Наталья Владимировна,NaN,1,2,3,4,5,NaN,NaN,NaN,NaN,NaN,5
44,Дяченко Мария Игоревна,NaN,1,2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3
59,Савченко Анастасия Игоревна,NaN,NaN,2,3,4,5,6,NaN,NaN,NaN,NaN,5
60,Глунчадзе Ираклий Вахтангович,NaN,NaN,2,3,4,5,NaN,NaN,NaN,NaN,NaN,4
83,Бекбулатов Рамзан Ломалиевич,NaN,NaN,NaN,3,4,5,6,NaN,NaN,NaN,NaN,4
87,Савин Павел Андреевич,NaN,NaN,NaN,3,4,NaN,6,NaN,NaN,NaN,NaN,3
88,Волчанский Андрей Петрович,NaN,NaN,NaN,3,4,NaN,NaN,NaN,NaN,NaN,NaN,2
89,Думаревский Александр Дмитриевич,NaN,NaN,NaN,3,4,NaN,NaN,NaN,NaN,NaN,NaN,2
91,Подкользина Евгения Борисовна,NaN,NaN,NaN,3,4,NaN,NaN,NaN,NaN,NaN,NaN,2
104,Чеботаева Виктория Алексеевна,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [ ]:
# научиться выделять не только однокурсников в текущем созыве, но и за все созывы
# (чтобы не зависеть от случайностей: например, в этот набор при фильтре по пятому созыву не попала бы Дяченко)

In [ ]:
# - по кандидату прочитать программы за все годы
kek = ssmm_membres_x_conv[['convocation_no','program_txt']][ssmm_membres_x_conv['name'] == 'Муравьёва Анна Сергеевна']
list(kek['program_txt'])

In [ ]:
# посмотреть, как падает конверсия из (учебного) года в год
# ЧЕРЕЗ: 
# разбить всех по году поступления (убрать академщиков и нестандартные кейсы)